In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - State


### Install adk

In [ ]:
%pip install --upgrade --quiet google-adk

In [ ]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [ ]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.

In [ ]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Session store

In [ ]:
import sqlite3
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content

APP_NAME = "Search_Assistant"
USER_ID = "shins"

# https://google.github.io/adk-docs/sessions/

if True: # InMemory
  session_service = InMemorySessionService()

else: # SQLite3
  db_url = "sqlite:///./mydatabase.db"
  session_service = DatabaseSessionService(db_url=db_url)

In [ ]:
session = session_service.create_session(app_name=APP_NAME,
                                         user_id=USER_ID,
                                         state={"initial_key": "initial_value"})

In [ ]:
print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {session.id}")
print(f"Application Name (`app_name`): {session.app_name}")
print(f"User ID (`user_id`):         {session.user_id}")
print(f"State (`state`):           {session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {session.last_update_time:.2f}")
print(f"---------------------------------")

--- Examining Session Properties ---
ID (`id`):                1be49897-8c33-4f1d-b5e0-51604949c0b0
Application Name (`app_name`): Search_Assistant
User ID (`user_id`):         shins
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1745749252.07
---------------------------------


In [ ]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.

"""

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Search_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      description="An assistant that can search the web.",
      tools=[google_search], # Use Google search tool
      output_key="last_data"
)

In [ ]:
from google.adk.runners import Runner

query = "What is the exchange rate from US dollars to Korean currency recently ?"

# Runner : An in-memory Runner for testing and development.
# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
runner = Runner(agent=agent,
                app_name=session.app_name,
                session_service=session_service)

# https://googleapis.github.io/python-genai/genai.html#genai.types.Content
content = types.Content(role='user', parts=[types.Part(text=query)])

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        pprint(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")


('Agent Response: As of recently, the exchange rate from US dollars (USD) to '
 'South Korean Won (KRW) is approximately 1 USD to 1437-1438 KRW.\n'
 '\n'
 'Here are a few examples of recent exchange rates:\n'
 '*   On Friday, April 25th, the rate was around 1438.49 KRW for 1 USD.\n'
 '*   Another recent quote shows 1 USD equaling 1437.42 KRW.\n'
 '*   The rate has been fluctuating, with sources showing slight variations '
 'around this number.\n'
 '\n'
 'Keep in mind that exchange rates change constantly due to market '
 "fluctuations. For the most up-to-the-minute rate, it's always best to check "
 'a reliable currency converter or financial news source at the time you plan '
 'to exchange currency.\n')


In [ ]:

updated_session = session_service.get_session(app_name = session.app_name, user_id = session.user_id, session_id = session.id)
pprint(f"State after agent run: {updated_session.state}")


("State after agent run: {'initial_key': 'initial_value', 'last_data': "
 '"As of recently, the exchange rate from US dollars (USD) to South Korean Won '
 '(KRW) is approximately 1 USD to 1437-1438 KRW.\\n\\nHere are a few examples '
 'of recent exchange rates:\\n*   On Friday, April 25th, the rate was around '
 '1438.49 KRW for 1 USD.\\n*   Another recent quote shows 1 USD equaling '
 '1437.42 KRW.\\n*   The rate has been fluctuating, with sources showing '
 'slight variations around this number.\\n\\nKeep in mind that exchange rates '
 'change constantly due to market fluctuations. For the most up-to-the-minute '
 "rate, it's always best to check a reliable currency converter or financial "
 'news source at the time you plan to exchange currency."}')


In [ ]:

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {updated_session.id}")
print(f"Application Name (`app_name`): {updated_session.app_name}")
print(f"User ID (`user_id`):         {updated_session.user_id}")
print(f"State (`state`):           {updated_session.state}")
print(f"Events (`events`):         {updated_session.events}")
print(f"Last Update (`last_update_time`): {updated_session.last_update_time:.2f}")
print(f"---------------------------------")


--- Examining Session Properties ---
ID (`id`):                1be49897-8c33-4f1d-b5e0-51604949c0b0
Application Name (`app_name`): Search_Assistant
User ID (`user_id`):         shins
State (`state`):           {'initial_key': 'initial_value', 'last_data': "As of recently, the exchange rate from US dollars (USD) to South Korean Won (KRW) is approximately 1 USD to 1437-1438 KRW.\n\nHere are a few examples of recent exchange rates:\n*   On Friday, April 25th, the rate was around 1438.49 KRW for 1 USD.\n*   Another recent quote shows 1 USD equaling 1437.42 KRW.\n*   The rate has been fluctuating, with sources showing slight variations around this number.\n\nKeep in mind that exchange rates change constantly due to market fluctuations. For the most up-to-the-minute rate, it's always best to check a reliable currency converter or financial news source at the time you plan to exchange currency."}
Events (`events`):         [Event(content=Content(parts=[Part(video_metadata=None, thought=None, 

## Add state manually

In [ ]:
print(f"Initial state: {session.state}")

Initial state: {'initial_key': 'initial_value'}


In [ ]:
import time

current_time = time.time()
state_changes = {
    "task_status": "active",              # Update session state
    "user:login_count": session.state.get("user:login_count", 0) + 1, # Update user state
    "user:last_login_ts": current_time,   # Add user state
    "temp:validation_needed": True        # Add temporary state (will be discarded)
}

In [ ]:
actions_with_update = EventActions(state_delta=state_changes)
# This event might represent an internal system action, not just an agent response
system_event = Event(
    invocation_id="inv_login_update",
    author="system", # Or 'agent', 'tool' etc.
    actions=actions_with_update,
    timestamp=current_time
    # content might be None or represent the action taken
)

# --- Append the Event (This updates the state) ---
session_service.append_event(session, system_event)
print("`append_event` called with explicit state delta.")

`append_event` called with explicit state delta.


In [ ]:
# --- Check Updated State ---
updated_session = session_service.get_session(app_name=session.app_name,
                                            user_id=session.user_id,
                                            session_id=session.id)
print(f"State after event: {updated_session.state}")

# Expected: {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': <timestamp>}
# Note: 'temp:validation_needed' is NOT present.

State after event: {'initial_key': 'initial_value', 'last_data': "As of recently, the exchange rate from US dollars (USD) to South Korean Won (KRW) is approximately 1 USD to 1437-1438 KRW.\n\nHere are a few examples of recent exchange rates:\n*   On Friday, April 25th, the rate was around 1438.49 KRW for 1 USD.\n*   Another recent quote shows 1 USD equaling 1437.42 KRW.\n*   The rate has been fluctuating, with sources showing slight variations around this number.\n\nKeep in mind that exchange rates change constantly due to market fluctuations. For the most up-to-the-minute rate, it's always best to check a reliable currency converter or financial news source at the time you plan to exchange currency.", 'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1745749255.7674565}


## End of notebook